In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

In [2]:
tf.__version__

'2.11.0'

## Create Binary Classification data with sklearn

In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


n = 50_000
d = 1_000


noise_factor = 0.01 # % of the labels are randomly flipped, DEFAULT=0.01
test_size = 0.2 # % of n
# The factor multiplying the hypercube size. Larger values spread out the 
# clusters/classes and make the classification task easier. DEFAULT=1
class_sep = -1
seed = None # None for no seed 7

# Create (noisy) testing data for binary classification.
X, y = make_classification(
    n_samples=n, 
    n_features=d,
    n_informative=d,
    n_redundant=0, 
    n_classes=2,
    class_sep=class_sep,
    flip_y=noise_factor,
    random_state=seed
)

# We will work with label values -1, +1 and not 0, +1 (convert)
y[y == 0] = -1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [4]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score

# PA-I regressor from sklearn
pa1 = PassiveAggressiveClassifier(C=0.01, loss="hinge", n_jobs=-1)
pa1.fit(X_train, y_train)

accuracy_score(y_test, pa1.predict(X_test))

0.7651

## Convert to Tensors

In [5]:
X_train_tensor = tf.constant(X_train, dtype=tf.float32)
y_train_tensor = tf.constant(y_train, dtype=tf.float32)
X_test_tensor = tf.constant(X_test, dtype=tf.float32)
y_test_tensor = tf.constant(y_test, dtype=tf.float32)

Delete sklearn type data 

In [6]:
del X, y, X_train, X_test, y_train, y_test

## Prepare data for Federated Learning

### Create centralized testing dataset

In [7]:
slices_test = (X_test_tensor, y_test_tensor)

In [8]:
def create_tf_dataset_for_testing(batch_size):
    return tf.data.Dataset.from_tensor_slices(slices_test).batch(batch_size)

In [9]:
test_dataset = create_tf_dataset_for_testing(32)

### Slice the Tensors for each Client

We will cut the training data, i.e., (`X_train_tensor`, `y_train_tensor`) to equal parts, each part corresponding to one Client. We want to give the result back as a dictionary with key `client_id` and value the training tensor data.

In [10]:
def create_data_for_clients(num_clients):
    
    client_slices_train = {}

    n_test = int(n - n*test_size)

    for i in range(num_clients):
        # Compute the indices for this client's slice
        start_idx = int(i * n_test / num_clients)
        end_idx = int((i + 1) * n_test / num_clients)

        # Get the slice for this client
        X_client_train = X_train_tensor[start_idx:end_idx]
        y_client_train = y_train_tensor[start_idx:end_idx]
        
        # Combine the slices into a single dataset
        client_slices_train[f'client_{i}'] = (X_client_train, y_client_train)
    
    return client_slices_train

### Create TF friendly data for each Client

Given a Tensor slice (i.e. value of `client_slices_train["client_id"]` we convert it to highly optimized `tf.data.Dataset` to prepare for training.

In [11]:
def create_tf_dataset_for_client(client_tensor_slices, batch_size, shuffle_buffer_size, num_steps_until_rtc_check, seed):
    
        return tf.data.Dataset.from_tensor_slices(client_tensor_slices) \
            .shuffle(buffer_size=shuffle_buffer_size, seed=seed).batch(batch_size) \
            .prefetch(tf.data.AUTOTUNE).take(num_steps_until_rtc_check)

### Create Federated Learning data

In [12]:
def create_federated_data(client_slices_train, batch_size, shuffle_buffer_size, num_steps_until_rtc_check, seed=None):
    
    federated_dataset = [ 
        create_tf_dataset_for_client(client_tensor_slices, batch_size, shuffle_buffer_size, num_steps_until_rtc_check, seed)
        for client, client_tensor_slices in client_slices_train.items()
    ]
    
    return federated_dataset

## PA-Classiers (binary classification)

![PA](images/PA_binary_classifiers.png)

In [13]:
@tf.function
def client_train(model, dataset, C):

    @tf.function
    def _train_on_batch(model, batch, C):

        x_batch, y_batch = batch
        
        # from shape (d,) make it (d,1)
        y_batch = tf.expand_dims(y_batch, axis=1)

        # dot(w, x) for the batch (each instance of x in x_batch) with with shape=(batchsize, 1)
        weights_dot_x_batch = tf.matmul(x_batch, model)

        # Prediction batch with shape=(batchsize, 1)
        y_pred_batch = tf.sign(weights_dot_x_batch)

        # Suffer loss for each prediction (of instance) in the batch with shape=(batchsize,1)
        loss_batch = tf.maximum(0., 1. - tf.multiply(y_batch, weights_dot_x_batch))

        # shape=(batchsize,1) where each instance is ||x||^2, x in x_batch
        norm_batch = tf.expand_dims(tf.reduce_sum(tf.square(x_batch), axis=1), axis=1)
        
        # PA-1 : Learning rate t for each instance x, with shape=(batchsize,1)
        t_batch = tf.maximum(C, tf.divide(loss_batch, norm_batch))

        # each instance is y*t*x, where y,t scalars and x in x_batch. shape=(batchsize,d)
        t_y_x_batch = tf.multiply(t_batch, tf.multiply(y_batch, x_batch))

        # !!!! Update with mean t*y*x
        t_y_x_update = tf.expand_dims(tf.reduce_mean(t_y_x_batch, axis=0) ,axis=1)

        # Update
        model.assign_add(t_y_x_update)
    
    for batch in dataset:
        _train_on_batch(model, batch, C)
    

# Functional Dynamic Averaging

We follow the Functional Dynamic Averaging (FDA) scheme. Let the mean model be

$$ \overline{w_t} = \frac{1}{k} \sum_{i=1}^{k} w_t^{(i)} $$

where $ w_t^{(i)} $ is the model at time $ t $ in some round in the $i$-th learner.

Local models are trained independently and cooperatively and we want to monitor the Round Terminating Conditon (**RTC**):

$$ \frac{1}{k} \sum_{i=1}^{k} \lVert w_t^{(i)} - \overline{w_t} \rVert_2^2  \leq \Theta $$

where the left-hand side is the **model variance**, and threshold $\Theta$ is a hyperparameter of the FDA, defined at the beginning of the round; it may change at each round. When the monitoring logic cannot guarantee the validity of RTC, the round terminates. All local models are pulled into `tff.SERVER`, and $\bar{w_t}$ is set to their average. Then, another round begins.

### Monitoring the RTC

FDA monitors the RTC by applying techniques from Functionary [Functional Geometric Averaging](http://users.softnet.tuc.gr/~minos/Papers/edbt19.pdf). We first restate the problem of monitoring RTC into the standard distributed stream monitoring formulation. Let

$$ S(t) =  \frac{1}{k} \sum_{i=1}^{k} S_i(t) $$

where $ S(t) \in \mathbb{R}^n $ be the "global state" of the system and $ S_i(t) \in \mathbb{R}^n $ the "local states". The goal is to monitor the threshold condition on the global state in the form $ F(S(t)) \leq \Theta $ where $ F : \mathbb{R}^n \to \mathbb{R} $ a non-linear function. Let

$$ \Delta_t^{(i)} = w_t^{(i)} - w_{t_0}^{(i)} $$

be the update at the $ i $-th learner, that is, the change to the local model at time $t$ since the beginning of the current round at time $t_0$. Let the average update be

$$ \overline{\Delta_t} = \frac{1}{k} \sum_{i=1}^{k} \Delta_t^{(i)} $$

it follows that the variance can be written as

$$ \frac{1}{k} \sum_{i=1}^{k} \lVert w_t^{(i)} - \overline{w_t} \rVert_2^2 = \Big( \frac{1}{k} \sum_{i=1}^{k} \lVert \Delta_t^{(i)} \rVert_2^2 \Big) - \lVert \overline{\Delta_t} \rVert_2^2 $$

So, conceptually, if we define
$$ S_i(t) = \begin{bmatrix}
           \lVert \Delta_t^{(i)} \rVert_2^2 \\
           \Delta_t^{(i)}
         \end{bmatrix} \quad \text{and} \quad
         F(\begin{bmatrix}
           v \\
           \bf{x}
         \end{bmatrix}) = v - \lVert \bf{x} \rVert_2^2 $$

The RTC is equivalent to condition $$ F(S(t)) \leq \Theta $$

## 2️⃣ Linear FDA

In the linear case, we reduce the update vector to a scalar, $ \xi \Delta_t^{(i)} \in \mathbb{R}$, where $ \xi $ is any unit vector.

Define the local state to be 

$$ S_i(t) = \begin{bmatrix}
           \lVert \Delta_t^{(i)} \rVert_2^2 \\
           \xi \Delta_t^{(i)}
         \end{bmatrix} \in \mathbb{R}^2 $$

Also, define 

$$ F(v, x) = v - x^2 $$

The RTC is equivalent to condition 

$$ F(S(t)) \leq \Theta $$

A random choice of $ \xi $ is likely to perform poorly (terminate round prematurely), as it wil likely be close to orthogonal to $ \overline{\Delta_t} $. A good choice would be a vector $ \xi $ correlated to $ \overline{\Delta_t} $. A heuristic choice is to take $ \overline{\Delta_{t_0}} $ (after scaling it to norm 1), i.e., the update vector right before the current round started. All nodes can estimate this without communication, as $ \overline{w_{t_0}} - \overline{w_{t_{-1}}} $, the difference of the last two models pushed by the Server. Hence, 

$$ \xi = \frac{\overline{w_{t_0}} - \overline{w_{t_{-1}}}}{\lVert \overline{w_{t_0}} - \overline{w_{t_{-1}}} \rVert_2} $$

In [14]:
@tf.function
def ksi_unit_fn(w_t0, w_tminus1):
    
    if tf.reduce_all(tf.equal(w_t0, w_tminus1)):
        # if equal then ksi becomes a random vector (will only happen in round 1)
        ksi = tf.random.normal(shape=w_t0.shape)
    else:
        ksi = w_t0 - w_tminus1

    # Normalize and return
    return tf.divide(ksi, tf.norm(ksi))


In [15]:
@tf.function
def steps_linear(model_tminus, model_t0, model, dataset, C):
    # number of steps depend on `.take()` from `dataset`
    client_train(model, dataset, C)
    
    Delta_i = model - model_t0
    
    #||D(t)_i||^2 , shape = (1,) 
    Delta_i_euc_norm_squared = tf.reduce_sum(tf.square(Delta_i), axis=0)
    
    # heuristic unit vector ksi
    ksi = ksi_unit_fn(model_t0, model_tminus)
    
    # ksi * Delta_i (* is dot) , shape = ()
    ksi_Delta_i = tf.reduce_sum(tf.multiply(ksi, Delta_i))
    
    return Delta_i_euc_norm_squared, ksi_Delta_i

## Accuracy Testing

In [16]:
@tf.function
def accuracy(model, dataset):
    
    @tf.function
    def _batch_accuracy(model, batch):
        x_batch, y_batch = batch
        # from shape (d,) make it (d,1)
        y_batch = tf.expand_dims(y_batch, axis=1)

        # dot(w, x) for the batch (each instance of x in x_batch) with with shape=(batchsize, 1)
        weights_dot_x_batch = tf.matmul(x_batch, model)

        # Prediction batch with shape=(batchsize, 1)
        y_pred_batch = tf.sign(weights_dot_x_batch)

        accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred_batch, y_batch), tf.float32))

        return accuracy
    
    # We take advantage of AutoGraph (convert Python code to TensorFlow-compatible graph code automatically)
    acc, num_batches = 0., 0.
    for batch in dataset:
        acc += _batch_accuracy(model, batch)
        num_batches += 1
        
    acc = acc / num_batches
    
    return acc

## Training Loop

In [17]:
def create_metrics_dict(fda_name, n, d, test_size, num_clients, batch_size, 
                        steps_in_one_fda_step, theta, c, total_fda_steps, 
                        total_rounds, final_accuracy, sketch_width=None, sketch_depth=None):
    metrics = {
            "fda_name" : fda_name,
            "theta" : theta,
            "n" : n,
            "d" : d,
            "test_size" : test_size,
            "num_clients" : num_clients,
            "batch_size" : batch_size,
            "steps_in_one_fda_step" : steps_in_one_fda_step,
            "sketch_width" : sketch_width,
            "sketch_depth" : sketch_depth,
            "c" : c
        }
    
    # one batch bytes
    metrics["one_sample_bytes"] = 4 * (metrics["d"] + 1)
    
    # training dataset size
    metrics["training_dataset_bytes"] = metrics["one_sample_bytes"] * (1 - metrics["test_size"]) * metrics["n"]
    
    # model bytes
    metrics["model_bytes"] = d * 4
    
    
    # local state bytes (i.e. S_i), for one client
    if fda_name == "naive":
        metrics["local_state_bytes"] = 4
    elif fda_name == "linear":
        metrics["local_state_bytes"] = 8
    else:
        metrics["local_state_bytes"] = sketch_width * sketch_depth * 4 + 4
        
    # accuracy (already computed in parameter)
    metrics["final_accuracy"] = final_accuracy
    
    # total fda steps from algo
    metrics["total_fda_steps"] = total_fda_steps
    
    # total steps (a single fda step might have many normal SGD steps, batch steps)
    metrics["total_steps"] = metrics["total_fda_steps"] * metrics["steps_in_one_fda_step"]
    
    # total rounds in algo. Reason why we differentiate from the hardcoded NUM_ROUNDS
    # is because we might run less rounds in the future (i.e. stop on 10^7 samples idk)
    metrics["total_rounds"] = total_rounds
    
    # bytes exchanged for synchronizing weights (x2 because server sends back)
    metrics["model_bytes_exchanged"] = metrics["total_rounds"] * metrics["model_bytes"] \
        * metrics["num_clients"] * 2
    
    # bytes exchanged for monitoring the variance (communication)
    metrics["monitoring_bytes_exchanged"] = metrics["local_state_bytes"] * metrics["total_fda_steps"] \
        * metrics["num_clients"]
    
    # total communication bytes (for both monitoring and model synchronization)
    metrics["total_communication_bytes"] = metrics["model_bytes_exchanged"] + metrics["monitoring_bytes_exchanged"]
    
    # total seen dataset bytes (across all learning, i.e., all clients)
    metrics["trained_in_bytes"] = metrics["batch_size"] * metrics["one_sample_bytes"] \
        * metrics["total_steps"] * metrics["num_clients"]
    
    return metrics

In [18]:
w_spec = tf.TensorSpec(shape=(20, d, 1), dtype=tf.float32)

@tf.function(input_signature=[w_spec, w_spec])
def variance(w_t, w_sync):
    # w_t , w_sync tensors with shape=(NUM_CLIENTS, d, 1)
    
    # tensor with shape=(NUM_CLIENTS, d, 1)
    diff = w_t - w_sync
    
    # tensor with shape=(NUM_CLIENTS, 1) , For each client ||w_i_t - w_t||^2
    dot = tf.reduce_sum(tf.square(diff), axis=1)
    
    # Variance shape=() , scalar
    var = tf.reduce_mean(dot)
    
    return var

In [19]:
import sys

In [44]:
def F_linear(S_1, S_2):
    return S_1 - S_2**2

In [48]:
@tf.function
def run_federated_simulation_linear(previous_server_model, server_model, client_models, federated_dataset, C, num_rounds, theta):
    
    print("retracing")
    
    total_rounds = 0
    total_fda_steps = 0
    
    S_1 = tf.constant(0., shape=(), dtype=tf.float32)
    S_2 = tf.constant(0., shape=(), dtype=tf.float32)
    
    for r in range(num_rounds):
        
        while F_linear(S_1, S_2) <= theta:
            euc_norm_squared_clients = []
            ksi_delta_clients = []

            # client steps (number depends on `federated_dataset`, i.e., `.take(num)`)
            for client_model, client_dataset in zip(client_models, federated_dataset):
                
                Delta_i_euc_norm_squared, ksi_Delta_i  = steps_linear(
                    previous_server_model, server_model, client_model, client_dataset, C
                )
                
                euc_norm_squared_clients.append(Delta_i_euc_norm_squared)
                ksi_delta_clients.append(ksi_Delta_i)
                
            S_1 = tf.reduce_mean(euc_norm_squared_clients)
            S_2 = tf.reduce_mean(ksi_delta_clients)
            
            total_fda_steps += 1
        
        # last server model (previous sync)
        previous_server_model.assign(server_model)
        
        """------------------------------test--------------------------------------------"""
        #Delta_i_clients = [tf.subtract(client_model, server_model) for client_model in client_models] #test
        #testing_approx_0 = tf.reduce_sum(tf.square(tf.reduce_mean(Delta_i_clients, axis=0)), axis=0) #test
        """------------------------------test--------------------------------------------"""
        
        # server average
        server_model.assign(tf.reduce_mean(client_models, axis=0))
        
        """------------------------------test--------------------------------------------"""
        actual_var = variance(client_models, [server_model]*len(client_models)) #test
        tf.print("Est left: ", S_1, " Est S_2: ", S_2**2, "Est var: ", S_1-S_2**2, " Actual var: ", actual_var, " Total fda steps: ", total_fda_steps, output_stream=sys.stdout)
        #tf.print("Est var: ", S_1-S_2, " Actual var: ", actual_var, " Missing: ", testing_approx_0, " Total fda steps: ", total_fda_steps, output_stream=sys.stdout)
        """------------------------------test--------------------------------------------"""
        
        
        # reset variance approx
        S_1 = tf.constant(0., shape=(), dtype=tf.float32)
        S_2 = tf.constant(0., shape=(), dtype=tf.float32)

        # synchronize clients
        for client_model in client_models:
            client_model.assign(server_model)
            
        total_rounds += 1
    
    return total_rounds, total_fda_steps

In [49]:
def run_tests():
    """ --------------- Test configurations -------------------"""
    
    NUM_CLIENTS = 20  # dataset
    BATCH_SIZE = 32 # dataset !
    NUM_STEPS_UNTIL_RTC_CHECK = 1 # dataset !  ALONE NO RETRACING
    NUM_ROUNDS = 10 # ! WE WANT !~~50~~!
    C = 0.01  # model, GOOD
    THETA = 100. # ALONE NO RETRACING (/ 10 of d)  WE WANT !~~100.~~!
    
    # Convert to tensors to avoid retracing where we can in `run_federated_simulation`
    c = tf.constant(C, shape=(), dtype=tf.float32)
    num_rounds = tf.constant(NUM_ROUNDS, shape=(), dtype=tf.int32)
    theta = tf.constant(THETA, shape=(), dtype=tf.float32)
    
    
    """ --------------- Initializations -------------------"""
    
    # 1. Dataset

    client_slices_train = create_data_for_clients(NUM_CLIENTS)

    federated_dataset = create_federated_data(
        client_slices_train=client_slices_train,
        batch_size=BATCH_SIZE,
        shuffle_buffer_size=int(n/20),
        num_steps_until_rtc_check=NUM_STEPS_UNTIL_RTC_CHECK,
        seed=seed
    )
    
    # 2. Models
    
    server_model = tf.Variable(tf.zeros(shape=(d, 1)), trainable=True, name='weights', dtype=tf.float32)
    
    # for `ξ` approximation.
    previous_server_model = tf.Variable(tf.zeros(shape=(d, 1)), trainable=True, name='weights', dtype=tf.float32)

    client_models = [
        tf.Variable(tf.zeros(shape=(d, 1)), trainable=True, name='weights', dtype=tf.float32)
        for _ in range(NUM_CLIENTS)
    ]
    
    """ --------------- Test -------------------"""
    
    c = tf.constant(C, shape=(), dtype=tf.float32)
    num_rounds = tf.constant(NUM_ROUNDS, shape=(), dtype=tf.int32)
    theta = tf.constant(THETA, shape=(), dtype=tf.float32)
    
    total_rounds, total_fda_steps = run_federated_simulation_linear(
        previous_server_model,
        server_model, 
        client_models, 
        federated_dataset, 
        c,
        num_rounds, 
        theta
    )
    
    final_accuracy = accuracy(server_model, test_dataset)
    
    metrics = create_metrics_dict(
        fda_name="linear", 
        n=n, 
        d=d, 
        test_size=test_size, 
        num_clients=NUM_CLIENTS, 
        batch_size=BATCH_SIZE, 
        steps_in_one_fda_step=NUM_STEPS_UNTIL_RTC_CHECK, 
        theta=THETA, 
        c=C, 
        total_fda_steps=total_fda_steps.numpy(), 
        total_rounds=total_rounds.numpy(), 
        final_accuracy=final_accuracy.numpy(), 
        sketch_width=None, 
        sketch_depth=None
    )
    
    return metrics

In [50]:
metrics = run_tests()

retracing
Est left:  100.837708  Est S_2:  0.00222611311 Est var:  100.83548  Actual var:  61.9981918  Total fda steps:  39
Est left:  166.342834  Est S_2:  65.936058 Est var:  100.406776  Actual var:  95.1428833  Total fda steps:  92
Est left:  165.646515  Est S_2:  64.9602432 Est var:  100.686272  Actual var:  95.8189545  Total fda steps:  145
Est left:  169.5784  Est S_2:  66.6086044 Est var:  102.969795  Actual var:  97.5546494  Total fda steps:  199
Est left:  170.323105  Est S_2:  68.0853195 Est var:  102.237785  Actual var:  96.7513275  Total fda steps:  253
Est left:  167.947067  Est S_2:  66.0359879 Est var:  101.911079  Actual var:  96.3050232  Total fda steps:  306
Est left:  164.681519  Est S_2:  64.2349701 Est var:  100.446548  Actual var:  95.0593567  Total fda steps:  359
Est left:  164.217392  Est S_2:  64.147728 Est var:  100.069664  Actual var:  94.6075  Total fda steps:  412
Est left:  168.502411  Est S_2:  67.6123428 Est var:  100.890068  Actual var:  95.4260712  To

In [51]:
metrics

{'fda_name': 'linear',
 'theta': 100.0,
 'n': 50000,
 'd': 1000,
 'test_size': 0.2,
 'num_clients': 20,
 'batch_size': 32,
 'steps_in_one_fda_step': 1,
 'sketch_width': None,
 'sketch_depth': None,
 'c': 0.01,
 'one_sample_bytes': 4004,
 'training_dataset_bytes': 160160000.0,
 'model_bytes': 4000,
 'local_state_bytes': 8,
 'final_accuracy': 0.8034145,
 'total_fda_steps': 519,
 'total_steps': 519,
 'total_rounds': 10,
 'model_bytes_exchanged': 1600000,
 'monitoring_bytes_exchanged': 83040,
 'total_communication_bytes': 1683040,
 'trained_in_bytes': 1329968640}

In [25]:
metrics["training_dataset_bytes"]/1_000_000

160.16

In [58]:
metrics["trained_in_bytes"]/1_000_000

1329.96864

In [27]:
import pandas as pd

In [28]:
lst = [metrics, metrics, metrics]

In [29]:
df = pd.DataFrame(lst)

In [30]:
df

,fda_name,theta,n,d,test_size,num_clients,batch_size,steps_in_one_fda_step,sketch_width,sketch_depth,...,model_bytes,local_state_bytes,final_accuracy,total_fda_steps,total_steps,total_rounds,model_bytes_exchanged,monitoring_bytes_exchanged,total_communication_bytes,trained_in_bytes
0,linear,10.0,50000,1000,0.2,20,32,1,None,None,...,4000,8,0.797424,44,44,5,800000,7040,807040,112752640
1,linear,10.0,50000,1000,0.2,20,32,1,None,None,...,4000,8,0.797424,44,44,5,800000,7040,807040,112752640
2,linear,10.0,50000,1000,0.2,20,32,1,None,None,...,4000,8,0.797424,44,44,5,800000,7040,807040,112752640


In [31]:
df.to_csv('test_results/naive.csv')

In [32]:
# Load DataFrame from a CSV file
df_from_csv = pd.read_csv('test_results/naive.csv')

In [33]:
df_from_csv

,Unnamed: 0,fda_name,theta,n,d,test_size,num_clients,batch_size,steps_in_one_fda_step,sketch_width,...,model_bytes,local_state_bytes,final_accuracy,total_fda_steps,total_steps,total_rounds,model_bytes_exchanged,monitoring_bytes_exchanged,total_communication_bytes,trained_in_bytes
0,0,linear,10.0,50000,1000,0.2,20,32,1,NaN,...,4000,8,0.797424,44,44,5,800000,7040,807040,112752640
1,1,linear,10.0,50000,1000,0.2,20,32,1,NaN,...,4000,8,0.797424,44,44,5,800000,7040,807040,112752640
2,2,linear,10.0,50000,1000,0.2,20,32,1,NaN,...,4000,8,0.797424,44,44,5,800000,7040,807040,112752640


1. check which changes produce new graph. We want 